1. 데이터 RDD 생성 map()
2. collect()
3. blue mnms만 꺼내기
4. 전체 합계 구하기
5. 도시별 합계 구하기

In [1]:
import sys
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

In [3]:
conf = SparkConf().setMaster("local[*]").setAppName("mnm-rdd-counts")

# SparkContext 생성
sc = SparkContext(conf=conf)

24/12/06 18:00:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
directory = os.path.join(os.getcwd(), "data")  # 현재 디렉토리의 data 폴더
filename  = "mnm_dataset.csv"
file_path = os.path.join(directory, filename)
lines = sc.textFile("file:///" + file_path.replace("\\", "/"))

In [5]:
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [11]:
#header 추출
header = lines.first()
data = lines.filter(lambda row: row != header)
print(header)
data.take(5)

State,Color,Count


['TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71', 'WA,Yellow,93']

In [13]:
mnms_data = data.map(lambda row: row.split(","))
mnms_data.take(5)

[['TX', 'Red', '20'],
 ['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WA', 'Yellow', '93']]

In [14]:
mnms_data.collect()[:3]

[['TX', 'Red', '20'], ['NV', 'Blue', '66'], ['CO', 'Blue', '79']]

In [18]:
#블루 컬러 구하기
blue_data = mnms_data.filter(lambda row: row[1]=="Blue")
blue_data.take(5)

[['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WY', 'Blue', '16'],
 ['AZ', 'Blue', '75']]

In [ ]:
#총 합계

In [20]:
count_sum = blue_data.map(lambda row: int(row[2])).sum()
print(f"Total Count: {count_sum}")

Total Count: 902516


In [21]:
#도시별 합계
city_rdd = blue_data.map(lambda row: (row[0], int(row[2])))
city_total_data = city_rdd.reduceByKey(lambda a,b: a+b)
print(city_total_data.collect())

[('CO', 93412), ('WY', 91002), ('CA', 89123), ('TX', 88466), ('NM', 90150), ('NV', 90003), ('OR', 90526), ('AZ', 89971), ('UT', 89977), ('WA', 89886)]


In [23]:
sc.stop()